## 基本配置　　
### 导入包和版本查询

In [5]:
import torch
import torch.nn as nn
import torchvision
print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print(torch.cuda.get_device_name(0))

1.4.0+cu100
10.0
7603
GeForce RTX 2080 Ti


### 可复现性
在硬件设备（CPU、GPU）不同时，完全的可复现性无法保证，即使随机种子相同。但是，在同一个设备上，应该保证可复现性。具体做法是，在程序开始的时候固定torch的随机种子，同时也把numpy的随机种子固定。

In [7]:
import numpy as np
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

### 显卡设置
如果只需要一张显卡

In [8]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

如果需要指定多张显卡，比如0，1号显卡。

In [9]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

也可以在命令行运行代码时设置显卡：  
`CUDA_VISIBLE_DEVICES=0,1 python train.py`  
清除缓存　　

In [10]:
torch.cuda.empty_cache()

### 张量(Tensor)处理

In [11]:
tensor = torch.randn(3,4,5)
print(tensor.type())  # 数据类型
print(tensor.size())  # 张量的shape，是个元组
print(tensor.dim())   # 维度的数量

torch.FloatTensor
torch.Size([3, 4, 5])
3


In [12]:
# Tensor[N, C, H, W]
images = torch.randn(32, 3, 56, 56)

In [15]:
images.sum(dim=1).shape

torch.Size([32, 56, 56])

In [17]:
NCHW = ['N', 'C', 'H', 'W']
images = torch.randn(32, 3, 56, 56, names=NCHW)

/pytorch/c10/core/TensorImpl.h:845: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable.


In [18]:
images.sum('C').shape

torch.Size([32, 56, 56])

In [19]:
images.select('C', index=0)

tensor([[[-9.9507e-01,  4.5089e-01,  1.4671e-01,  ...,  3.4687e-01,
          -2.9055e-01, -1.7921e+00],
         [-5.8186e-01, -2.5670e-01,  7.5862e-01,  ..., -3.1995e-02,
          -4.1721e-01, -7.8874e-01],
         [ 1.1152e+00, -1.1732e+00,  1.7059e-01,  ...,  9.0192e-01,
           9.1724e-01,  8.2328e-01],
         ...,
         [ 1.2963e+00, -9.4490e-01,  5.7915e-01,  ..., -8.5010e-01,
          -1.0993e+00,  8.6259e-01],
         [-2.2042e-01,  3.9036e-01, -9.8582e-02,  ..., -1.8150e-01,
           9.1908e-02,  4.3104e-02],
         [ 1.9219e+00,  2.1617e+00, -6.1160e-01,  ...,  1.5528e-01,
          -1.9943e+00, -1.0316e+00]],

        [[-2.3882e+00,  1.0923e+00,  6.3999e-01,  ..., -9.5147e-01,
           4.0850e-02,  2.7926e-01],
         [-7.1295e-01,  1.1391e-01,  5.9903e-01,  ...,  1.7090e-01,
           5.9664e-01, -3.2352e-01],
         [-1.7379e+00, -1.2598e+00,  2.7185e-01,  ..., -1.0333e+00,
          -2.4391e-01, -2.1693e-01],
         ...,
         [ 8.2945e-01, -1

In [20]:
tensor = torch.rand(3,4,1,2,names=('C', 'N', 'H', 'W'))

In [23]:
tensor.shape

torch.Size([3, 4, 1, 2])

In [24]:
tensor = tensor.align_to('N', 'C', 'H', 'W')

In [25]:
tensor.shape

torch.Size([4, 3, 1, 2])

### torch.Tensor与np.ndarray转换
除了CharTensor，其他所有CPU上的tensor都支持转换为numpy格式然后再转换回来。

In [54]:
img_1 = np.random.randn(3,25,25)
img_2 = torch.from_numpy(img).float()

In [55]:
img_1.dtype

dtype('float64')

In [56]:
img_1.shape

(3, 25, 25)

In [57]:
img_2.type

<function Tensor.type>

In [58]:
img_3 = img_2.cpu().numpy()

In [59]:
img_3.dtype

dtype('float32')

In [60]:
img_3.shape

(3, 25, 25)

### np.ndarray 与 PIL.Image的转换

In [61]:
img_3.astype(np.uint8)

array([[[  0,   0, 253, ...,   0,   0,   0],
        [  0,   1,   0, ...,   0, 255,   1],
        [  0,   0, 255, ...,   1,   0,   0],
        ...,
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   1,   2, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0, 255]],

       [[  1,   0, 255, ..., 254,   0,   1],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   1, ..., 255,   0,   0],
        ...,
        [  0,   0,   0, ...,   0,   1,   0],
        [  1,   0,   1, ...,   0,   1,   0],
        [  1, 255,   0, ...,   0,   0,   0]],

       [[  0,   0, 255, ..., 254, 254,   0],
        [  1,   0,   0, ...,   0,   0,   0],
        [  0, 255,   1, ...,   0,   0,   0],
        ...,
        [  0,   1,   1, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0, 255,   0],
        [  0,   0,   1, ...,   0,   0, 255]]], dtype=uint8)

In [62]:
import PIL

In [63]:
image = PIL.Image.fromarray(img_3.astype(np.uint8))

TypeError: Cannot handle this data type: (1, 1, 25), |u1

In [ ]:
ndarray = np.asarray(PIL.Image.open(path))

In [64]:
value = torch.rand(1).item()

### 数据类型转换

In [26]:
# 设置默认类型，pytorch中的FloatTensor远远快于DoubleTensor
torch.set_default_tensor_type(torch.FloatTensor)

In [27]:
# 类型转换
tensor = tensor.cuda()
tensor.dtype

torch.float32

In [28]:
tensor = tensor.cpu()
tensor.dtype

torch.float32

In [29]:
tensor = tensor.float()
tensor.dtype

torch.float32

In [30]:
tensor = tensor.long()
tensor.dtype

torch.int64

## Tensor

本质上来说，PyTorch 是一个处理张量的库。一个张量是一个数字、向量、矩阵或任何 n 维数组。

In [1]:
import torch

# number
t1 = torch.tensor(4.)

In [2]:
t1

tensor(4.)

In [3]:
t1.dtype

torch.float32

In [4]:
# vector
t2 = torch.tensor([1, 2, 3, 4])
t2

tensor([1, 2, 3, 4])

In [ ]:
# Matrix
t3 = torch.tensor([[5., 6]])